<a href="https://colab.research.google.com/github/rahiakela/natural-language-processing-case-studies/blob/master/bert-mechanism/create_and_deploy_simple_sentiment_analysis_via_fastapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Create and Deploy a Simple Sentiment Analysis via FastAPI

For this notebook, we will skip the building of our own model, and instead leverage the Pipeline class of the HuggingFace Transformers library. Transformers is full of SOTA NLP models which can be used out of the box as-is, as well as fine-tuned for specific uses and high performance.

Using the Transformers library, FastAPI, and astonishingly little code, we are going to create and deploy a very simple sentiment analysis app.

##Setup

In [ ]:
%%shell

pip install transformers
pip install fastapi
pip install uvicorn
pip install nest-asyncio 
pip install pyngrok

In [5]:
from transformers import pipeline
from fastapi import FastAPI

import nest_asyncio
from pyngrok import ngrok
import uvicorn

## Sentiment Analysis API

For us, the task is `sentiment-analysis` and the model is `nlptown/bert-base-multilingual-uncased-sentiment`. This is a BERT model trained for multilingual sentiment analysis, and which has been contributed to the HuggingFace model repository by NLP Town.

In [ ]:
nlp = pipeline(task="sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

In [10]:
app = FastAPI()

@app.get("/")
def get_root():
  return {"message": "This is the sentiment analysis app"}

@app.get("/sentiment")
def query_sentiment(text: str):
  return analyze_sentiment(text)


def analyze_sentiment(text: str):
  result = nlp(text)

  sent = ""
  if result[0]["label"] == "1 star":
    sent = "very negative"
  elif  result[0]["label"] == "2 star":
    sent = "negative"
  elif  result[0]["label"] == "3 star":
    sent = "neutral"
  elif  result[0]["label"] == "4 star":
    sent = "positive"
  else:
    sent = "very positive"

  prob = result[0]["score"]

  return {"sentiment": sent, "probability": prob}

And now we have a REST API capable of accepting get requests and performing sentiment analysis. Before we try it out, we have to run the Uvicorn web server which will provide the necessary back end functionality.

In [11]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://c17abd88f6ae.ngrok.io


INFO:     Started server process [60]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     223.190.85.56:0 - "GET /sentiment?text=welcome%20to%20my%20home! HTTP/1.1" 200 OK
INFO:     223.190.85.56:0 - "GET /sentiment?text=i%20don%27t%20like%20your%20cat HTTP/1.1" 200 OK
INFO:     2600:1f18:65b9:df01:faa1:9722:e9eb:e34b:0 - "GET /sentiment?text=welcome%20to%20my%20home! HTTP/1.1" 200 OK
INFO:     223.190.85.56:0 - "GET /sentiment?text=that%20movie%20was%20just%20okay HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [60]
